<a href="https://colab.research.google.com/github/jberardini2/Baseball-Stats/blob/main/M6_Assignment_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 Summary and Questions


I work for a technology reseller and the leadership team has asked to report back on sales data for the different companies my company promotes products for. The leadership team is interested in reviewing:

The average sales per company

The count of sales per company

The maximum sales amount per company

The minimum sales amount per company

The cumulative sales per company

The maximum sales amount across companies

The distinct count of sales across companies

The standard deviation of the sales across companies

An ordering of the sales amounts from the maximum amount to the minimum amount

# Setting up PySpark and creating a Spark Session

In [2]:
# Update environment (if needed)
# !sudo apt update

# Download and install Java
!sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [3]:
# Install Apache Spark 3.2.1 with Hadoop 3.2
!wget -nc -q https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

In [4]:
# Unzip the folder
!tar xf  spark-3.2.1-bin-hadoop3.2.tgz

In [5]:
# Install findspark library that will locate Spark on the system
!pip install -q findspark

In [6]:
# Setting the environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [7]:
import findspark
findspark.init()

In [8]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# After executing the cell above, Drive
# files will be present in "/content/My Drive".
!ls "/content/drive/My Drive/Education/Big Data"

 acite75_99.zip
 airline.csv
 books.csv
'M1-First-Class (1).mp4'
 M1-First-Class.mp4
'M2 Discussion - Hadoop.gdoc'
'M2-Hadoop-Lecture (1).mp4'
 M2-Hadoop-Lecture.mp4
'M2 - Introduction to Hadoop.gdoc'
'M3 - Advanced MR Concepts and Hadoop 2.0.gdoc'
'M3 Assignment - YARN.gdoc'
'M3-Map Reduce and Joins (1).mp4'
'M3-Map Reduce and Joins.mp4'
'M4 Assignment - HIVE with Google Cloud.gdoc'
'M4 - Class Notes for Hive.gdoc'
'M4-Google Cloud - uploading to google storage and then copying file using the gsutil command to your VM.JPG'
'M4-HADOOP- Example of loading data from local file system into Hadoop.JPG'
'M4-HADOOP- Querying for database and table directories.JPG'
'M4-HADOOP-Using put to copy from local linux to HDFS location under user directory.JPG'
'M4-HIVE-Couting rows.JPG'
'M4-HIVE-Creating a Database.JPG'
'M4-HIVE-creating defined but empty table.JPG'
'M4-HIVE-describe formatted to view text file versus RC file.JPG'
'M4-HIVE-Example of joining 2 tables with count-group-orderby.JPG'
'M4

In [10]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

# Create spark session
spark = SparkSession.builder.getOrCreate()

spark.version

'3.2.1'

In [11]:
df = spark.read.option("header", "true").csv("/content/drive/My Drive/Education/Big Data/Sales Info.csv")

In [12]:
df.show()

+-------+---------+-----+
|Company|   Person|Sales|
+-------+---------+-----+
|   ADBE|     Nick|  400|
|   ADBE|  Charles|  240|
|   ADBE|    Frank|  680|
|   SFDC|     Jody| 1200|
|   SFDC|      Lee|  248|
|   SFDC|  Hui-Ann|  286|
|   AMZN|     Dale| 1720|
|   AMZN|Christine|  700|
|    SAP|     Jill|  500|
|    SAP|      Tim|  260|
|    SAP|    Danny| 1500|
|    SAP|    Peter|  700|
|   ORCL|    Brian|  450|
|   ORCL|  Randolp|  290|
|   ORCL|   Florin|  720|
|   ORCL|    James| 1250|
|   ORCL|     Eric| 1400|
|   BABA|   Darren|  350|
|   BABA|   Derick|  475|
|   BABA|   Damien|  585|
+-------+---------+-----+
only showing top 20 rows



In [13]:
#Display the data types of the columns

df.dtypes

[('Company', 'string'), ('Person', 'string'), ('Sales', 'string')]

Register the DataFrame as a SQL temporary view so that you can interact with it using SQL commands


In [14]:
df.createOrReplaceTempView("sales")

Query the temporary view using SQL

In [15]:
sqlDF = spark.sql("SELECT * FROM sales")
sqlDF.show()

+-------+---------+-----+
|Company|   Person|Sales|
+-------+---------+-----+
|   ADBE|     Nick|  400|
|   ADBE|  Charles|  240|
|   ADBE|    Frank|  680|
|   SFDC|     Jody| 1200|
|   SFDC|      Lee|  248|
|   SFDC|  Hui-Ann|  286|
|   AMZN|     Dale| 1720|
|   AMZN|Christine|  700|
|    SAP|     Jill|  500|
|    SAP|      Tim|  260|
|    SAP|    Danny| 1500|
|    SAP|    Peter|  700|
|   ORCL|    Brian|  450|
|   ORCL|  Randolp|  290|
|   ORCL|   Florin|  720|
|   ORCL|    James| 1250|
|   ORCL|     Eric| 1400|
|   BABA|   Darren|  350|
|   BABA|   Derick|  475|
|   BABA|   Damien|  585|
+-------+---------+-----+
only showing top 20 rows



# Using SQL to interface with Spark

## The average sales per company


In [16]:
#Convert the Sales column to a numeric value and name it Average Sales. 
sqlDF = sqlDF.withColumn("Average Sales", sqlDF.Sales.cast('Integer'))
#Group by Company and calculate average sales.
sqlDF.groupBy("Company").avg("Average Sales").show()


+-------+------------------+
|Company|avg(Average Sales)|
+-------+------------------+
|   SFDC|             578.0|
|   ORCL|             822.0|
|    SAP|             740.0|
|   AMZN|            1210.0|
|   ADBE|             440.0|
|   BABA|             600.0|
+-------+------------------+



## The count of sales per company


In [17]:
#Group by Company and count how many sales.
sqlDF.groupBy("Company").count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   SFDC|    3|
|   ORCL|    5|
|    SAP|    4|
|   AMZN|    2|
|   ADBE|    3|
|   BABA|    4|
+-------+-----+



## The maximum sales amount per company


In [18]:
#Convert the Sales column to a numeric value and name it I_Sales. 
sqlDF = sqlDF.withColumn("I_Sales", sqlDF.Sales.cast('Integer'))
#Group by Company and calculate max sales per company.
sqlDF.groupBy("Company").max("I_Sales").show()

+-------+------------+
|Company|max(I_Sales)|
+-------+------------+
|   SFDC|        1200|
|   ORCL|        1400|
|    SAP|        1500|
|   AMZN|        1720|
|   ADBE|         680|
|   BABA|         990|
+-------+------------+



## The minimum sales amount per company

In [19]:
#Convert the Sales column to a numeric value and name it I_Sales. 
sqlDF = sqlDF.withColumn("I_Sales", sqlDF.Sales.cast('Integer'))
#Group by Company and calculate min sales per company.
sqlDF.groupBy("Company").min("I_Sales").show()

+-------+------------+
|Company|min(I_Sales)|
+-------+------------+
|   SFDC|         248|
|   ORCL|         290|
|    SAP|         260|
|   AMZN|         700|
|   ADBE|         240|
|   BABA|         350|
+-------+------------+



## The cumulative sales per company

In [20]:
#Convert the Sales column to a numeric value and name it Count of Sales. 
sqlDF = sqlDF.withColumn("Count of Sales", sqlDF.Sales.cast('Integer'))
#Group by Company and sum sales per company.
sqlDF.groupBy("Company").sum("Count of Sales").show()

+-------+-------------------+
|Company|sum(Count of Sales)|
+-------+-------------------+
|   SFDC|               1734|
|   ORCL|               4110|
|    SAP|               2960|
|   AMZN|               2420|
|   ADBE|               1320|
|   BABA|               2400|
+-------+-------------------+



In [21]:
 sqlDF = sqlDF.withColumn('Sales', sqlDF['Sales'].cast('Integer'))
 sqlDF.groupBy().sum('Sales').show()

 df2 = df.withColumn('Sales', df['Sales'].cast('float'))
 df2.groupBy().sum('Sales').show()
 

+----------+
|sum(Sales)|
+----------+
|     14944|
+----------+

+----------+
|sum(Sales)|
+----------+
|   14944.0|
+----------+



## The maximum sales amount across companies


In [22]:

# The highest sales amount for all companies is 1720 by AMZN.
sqlDF = sqlDF.withColumn("I_Sales", sqlDF.Sales.cast('Integer'))
sqlDF.groupBy().max("I_Sales").show()


+------------+
|max(I_Sales)|
+------------+
|        1720|
+------------+



## The distinct count of sales across companies


In [23]:
# Group by Company and then do a count followed by distinct.
sqlDF.groupBy("Company").count().distinct().show()




+-------+-----+
|Company|count|
+-------+-----+
|   SFDC|    3|
|   ORCL|    5|
|    SAP|    4|
|   AMZN|    2|
|   ADBE|    3|
|   BABA|    4|
+-------+-----+



## The standard deviation of the sales across companies


In [24]:
# Import pyspark.sql.functions as F
import pyspark.sql.functions as F

sqlDF = sqlDF.withColumn("I_Sales", sqlDF.Sales.cast('Integer'))
# Standard deviation of sales by company
sqlDF.groupBy("Company").agg(F.stddev("I_Sales")).show()

+-------+--------------------+
|Company|stddev_samp(I_Sales)|
+-------+--------------------+
|   SFDC|   539.0027829241701|
|   ORCL|    487.103685060994|
|    SAP|   537.6492040974921|
|   AMZN|   721.2489168102785|
|   ADBE|  222.71057451320087|
|   BABA|    277.158197906298|
+-------+--------------------+



## An ordering of the sales amounts from the maximum amount to the minimum amount

In [69]:
# To sort descending its necessary to use the "col" function. According to documentation orderBy gives ascending only.
from pyspark.sql.functions import col

#sqlDF = sqlDF.withColumn("I_Sales", sqlDF.Sales.cast('Integer'))

sqlDF = sqlDF.drop("Average Sales","Count of Sales","Sales")

print ("2 diferent ways to sort")
sqlDF = sqlDF.orderBy('I_Sales', ascending=False)
sqlDF.show()
sqlDF = sqlDF.orderBy(col("I_Sales").desc())
sqlDF.show()



2 diferent ways to sort
+-------+---------+-------+
|Company|   Person|I_Sales|
+-------+---------+-------+
|   AMZN|     Dale|   1720|
|    SAP|    Danny|   1500|
|   ORCL|     Eric|   1400|
|   ORCL|    James|   1250|
|   SFDC|     Jody|   1200|
|   BABA|    David|    990|
|   ORCL|   Florin|    720|
|    SAP|    Peter|    700|
|   AMZN|Christine|    700|
|   ADBE|    Frank|    680|
|   BABA|   Damien|    585|
|    SAP|     Jill|    500|
|   BABA|   Derick|    475|
|   ORCL|    Brian|    450|
|   ADBE|     Nick|    400|
|   BABA|   Darren|    350|
|   ORCL|  Randolp|    290|
|   SFDC|  Hui-Ann|    286|
|    SAP|      Tim|    260|
|   SFDC|      Lee|    248|
+-------+---------+-------+
only showing top 20 rows

+-------+---------+-------+
|Company|   Person|I_Sales|
+-------+---------+-------+
|   AMZN|     Dale|   1720|
|    SAP|    Danny|   1500|
|   ORCL|     Eric|   1400|
|   ORCL|    James|   1250|
|   SFDC|     Jody|   1200|
|   BABA|    David|    990|
|   ORCL|   Florin|    720

In [26]:
# Print the date and time of when homework was worked on.
import datetime
import pytz

datetime.datetime.now(pytz.timezone('US/Central')).strftime("%a, %d %B %Y %H:%M:%S")

'Fri, 03 June 2022 12:44:24'